# DIC Crack Inspector

In [1]:
%matplotlib widget

In [2]:
from bmcs_shear.dic_crack import DICGrid, DICAlignedGrid, DICCOR, DICInspector, DICCrack

Convenience function to construct cracks

In [3]:
def construct_dic_cracks(cracks):
    '''Use the supplied list of crack specifications to construct the DICCrack objects'''
    dic_cracks = []
    for i, (ref_frame, rot_grid) in enumerate(cracks):
        dic_aligned_grid = DICAlignedGrid(dic_grid=dic_grid)
        dic_aligned_grid.trait_set(**ref_frame)
        dic_cor = DICCOR(dic_aligned_grid=dic_aligned_grid, crack_idx=i+1)
        dic_cor.trait_set(**rot_grid)
        dic_crack = DICCrack(name='crack #%d' % (i+1), dic_cor=dic_cor)
        dic_cracks.append(dic_crack)
    return dic_cracks

# B1_TV1

In [4]:
import numpy as np
phi = np.array([np.nan, 5, 6])
print(phi)
phi = np.where(np.isnan(phi), 0, phi)
phi

[nan  5.  6.]


array([0., 5., 6.])

In [5]:
B1_TV1_cracks = [
    # crack 1
    (dict(y_ref_i=-1, y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min = -7, n_x_max = -6, n_x_step=1, n_y_min = 1, n_y_max=-2, n_y_step = 1)),
    # crack 2
    (dict(y_ref_i=-7, y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min = -13, n_x_max = -10, n_x_step=1, n_y_min = 1, n_y_max=-1, n_y_step = 1)),
    # crack 3
    (dict(y_ref_i=-11, y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min = -20, n_x_max = -17, n_x_step=1, n_y_min = 1, n_y_max=7, n_y_step = 1)),
    # crack 4
    (dict(y_ref_i=-20, y_ref_j_min=1, y_ref_j_max=10),
    dict(n_x_min = 29, n_x_max = 37, n_x_step=1, n_y_min = 7, n_y_step = 1)),
    # crack 5
    (dict(y_ref_i=-28, y_ref_j_min=1, y_ref_j_max=10),
    dict(n_x_min=-35, n_x_max=-32, n_x_step=1, n_y_min = 2, n_y_max=12, n_y_step = 1)),
    # crack 6
    (dict(y_ref_i=-33, y_ref_j_min=1, y_ref_j_max=10),
    dict(n_x_min=-46, n_x_max=-39, n_x_step=2, n_y_min = 6, n_y_max=-1, n_y_step = 2)),
    # crack 7
    (dict(y_ref_i=13, y_ref_j_min=1, y_ref_j_max=10),
    dict(n_x_min=1, n_x_max=8, n_x_step=2, n_y_min = 2, n_y_max=10, n_y_step = 2)),
]

In [6]:
dic_grid = DICGrid(n_x=59, n_y=15, d_x = 22, d_y = 22,  start_t = 0, end_t =31, U_factor = 1, 
                   dir_name='B1_TV1', grid_column_first = False, grid_number_vertical = False)
dic_insp = DICInspector(dic_grid=dic_grid)
dic_insp.dic_cracks.items = construct_dic_cracks(B1_TV1_cracks)
dic_insp.interact()

c:\users\fahad\pycharmprojects\bmcs_shear_zone\bmcs_shear\dic_crack\dic_cor.py:142: RuntimeWarning: invalid value encountered in true_divide
  phi = 2 * np.arctan(len_d_0t / len_d_0c)


In [7]:
dic_cor = dic_insp.dic_cracks.items[1].dic_cor

In [8]:
#dic_cor.dic_aligned_grid.X_ref_a

In [9]:
dic_cor.X_cor, dic_cor.X_cor_b

(array([114.1775775 , 211.06035055]), array([114.6137369 , 211.09543354]))

# B1_TV2

In [10]:
B1_TV2_cracks = [
    # crack 1
    (dict(y_ref_i=-1, y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min = -7, n_x_max = -6, n_x_step=1, n_y_min = 1, n_y_max=-5, n_y_step = 1)),
    # crack 2
    (dict(y_ref_i=-7, y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min = -16, n_x_max = -12, n_x_step=1, n_y_min = 7, n_y_max=-1, n_y_step = 1)),
    # crack 3
    (dict(y_ref_i=-23, y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min = -31, n_x_max = -26, n_x_step=1, n_y_min = 1, n_y_max=7, n_y_step = 1)),
    # crack 4
    (dict(y_ref_i=-31, y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min = 20, n_x_max = 24, n_x_step=1, n_y_min = 7, n_y_step = 1)),
    # crack 5
    (dict(y_ref_i=15, y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min=5, n_x_max=9, n_x_step=1, n_y_min = 7, n_y_max=12, n_y_step = 1)),
]

In [11]:
dic_grid = DICGrid(n_x=57, n_y=15, d_x = 22, d_y = 22,  start_t = 0, end_t =27, U_factor = 1, 
                   dir_name='B1_TV2', grid_column_first = False, grid_number_vertical = False)
dic_insp = DICInspector(dic_grid=dic_grid)
dic_insp.dic_cracks.items = construct_dic_cracks(B1_TV2_cracks)
dic_insp.interact()

c:\users\fahad\pycharmprojects\bmcs_shear_zone\bmcs_shear\dic_crack\dic_cor.py:142: RuntimeWarning: invalid value encountered in true_divide
  phi = 2 * np.arctan(len_d_0t / len_d_0c)


In [12]:
dic_grid.load_levels

array([  0.   ,  11.   ,  27.   ,  37.   ,  53.   ,  64.   ,  74.   ,
        84.   , 100.   , 116.   , 127.   , 138.   , 151.   , 161.5  ,
       175.   , 173.136, 170.93 , 173.6  , 185.351, 193.255, 195.103,
       195.381, 195.842, 196.132, 190.4  , 186.732, 185.528, 173.   ])

# B5_TV1

In [4]:
B5_TV1_cracks = [
    # crack 1
    (dict(y_ref_i=-1, y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min = -7, n_x_max = -6, n_x_step=1, n_y_min = 1, n_y_max=-5, n_y_step = 1)),
    # crack 2
    (dict(y_ref_i=-7, y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min = -12, n_x_max = -11, n_x_step=1, n_y_min = 1, n_y_max=-5, n_y_step = 1)),
    # crack 3
    (dict(y_ref_i=-12, y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min = -18, n_x_max = -16, n_x_step=1, n_y_min = 1, n_y_max=7, n_y_step = 1)),
    # crack 4
    (dict(y_ref_i=-18, y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min = -23, n_x_max = -21, n_x_step=1, n_y_min = 5, n_y_step = 1)),
    # crack 5
    (dict(y_ref_i=-23, y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min=-30, n_x_max=-27, n_x_step=1, n_y_min = 1, n_y_max=8, n_y_step = 1)),
    # crack 6
    (dict(y_ref_i=-30, y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min=-37, n_x_max=-34, n_x_step=1, n_y_min = 1, n_y_max=8, n_y_step = 1)),
    # crack 7
    (dict(y_ref_i=-37, y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min=-46, n_x_max=-42, n_x_step=1, n_y_min = 5, n_y_max=12, n_y_step = 1)),
    # crack 8
    (dict(y_ref_i=-44, y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min=-52, n_x_max=-50, n_x_step=1, n_y_min = 5, n_y_max=12, n_y_step = 1)),
    # crack 9
    (dict(y_ref_i=-52, y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min=-64, n_x_max=-58, n_x_step=1, n_y_min = 1, n_y_max=9, n_y_step = 1)),
]

In [25]:
dic_grid = DICGrid(n_x=68, n_y=11, d_x = 25, d_y = 25,  start_t = 0, end_t =23, U_factor = 1, 
                   dir_name='B5_TV1', grid_column_first = True, grid_number_vertical = True)
dic_aligned_grid = DICAlignedGrid(dic_grid=dic_grid,
                                 y_ref_i=-1, y_ref_j_min=1, y_ref_j_max=10)
dic_cor = DICCOR(dic_aligned_grid=dic_aligned_grid,
                 n_x_min = -7, n_x_max = -6, n_x_step=1, n_y_min = 1, n_y_max=-5, n_y_step = 1)
dic_cor.interact()

c:\users\fahad\pycharmprojects\bmcs_shear_zone\bmcs_shear\dic_crack\dic_cor.py:142: RuntimeWarning: invalid value encountered in true_divide
  phi = 2 * np.arctan(len_d_0t / len_d_0c)


In [5]:
dic_grid = DICGrid(n_x=68, n_y=11, d_x = 25, d_y = 25,  start_t = 0, end_t =23, U_factor = 1, 
                   dir_name='B5_TV1', grid_column_first = True, grid_number_vertical = True)
dic_insp = DICInspector(dic_grid=dic_grid)
dic_insp.dic_cracks.items = construct_dic_cracks(B5_TV1_cracks)
dic_insp.interact()

c:\users\fahad\pycharmprojects\bmcs_shear_zone\bmcs_shear\dic_crack\dic_cor.py:142: RuntimeWarning: invalid value encountered in true_divide
  phi = 2 * np.arctan(len_d_0t / len_d_0c)
c:\users\fahad\pycharmprojects\bmcs_shear_zone\bmcs_shear\dic_crack\dic_cor.py:142: RuntimeWarning: invalid value encountered in true_divide
  phi = 2 * np.arctan(len_d_0t / len_d_0c)
c:\users\fahad\pycharmprojects\bmcs_shear_zone\bmcs_shear\dic_crack\dic_cor.py:142: RuntimeWarning: invalid value encountered in true_divide
  phi = 2 * np.arctan(len_d_0t / len_d_0c)
c:\users\fahad\pycharmprojects\bmcs_shear_zone\bmcs_shear\dic_crack\dic_cor.py:142: RuntimeWarning: invalid value encountered in true_divide
  phi = 2 * np.arctan(len_d_0t / len_d_0c)


# B5_TV2

In [6]:
B5_TV2_cracks = [
    # crack 1
    (dict(y_ref_i=-1, y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min = -4, n_x_max = -3, n_x_step=1, n_y_min = 1, n_y_max=6, n_y_step = 1)),
    # crack 2
    (dict(y_ref_i=-4, y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min = -13, n_x_max = -11, n_x_step=1, n_y_min = 5, n_y_max=9, n_y_step = 1)),
    # crack 3
    (dict(y_ref_i=-13, y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min = -18, n_x_max = -16, n_x_step=1, n_y_min = 5, n_y_max=9, n_y_step = 1)),
    # crack 4
    (dict(y_ref_i=-18, y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min = -26, n_x_max = -23, n_x_step=1, n_y_min = 5, n_y_step = 1)),
    # crack 5
    (dict(y_ref_i=-26, y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min=-31, n_x_max=-28, n_x_step=1, n_y_min = 3, n_y_max=10, n_y_step = 1)),
    # crack 6
    (dict(y_ref_i=-31, y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min=-37, n_x_max=-35, n_x_step=1, n_y_min = 4, n_y_max=11, n_y_step = 1)),
    # crack 7
    (dict(y_ref_i=-37, y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min=-46, n_x_max=-42, n_x_step=1, n_y_min = 5, n_y_max=10, n_y_step = 1)),
    # crack 8
    (dict(y_ref_i=-46, y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min=-51, n_x_max=-48, n_x_step=1, n_y_min = 5, n_y_max=10, n_y_step = 1)),
    # crack 9
    (dict(y_ref_i=-52, y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min=-64, n_x_max=-58, n_x_step=1, n_y_min = 2, n_y_max=9, n_y_step = 1)),
]

In [7]:
dic_grid = DICGrid(n_x=71, n_y=12, d_x = 25, d_y = 25,  start_t = 0, end_t =20, U_factor = 1, 
                   dir_name='B5_TV2', grid_column_first = True, grid_number_vertical = True)
dic_insp = DICInspector(dic_grid=dic_grid)
dic_insp.dic_cracks.items = construct_dic_cracks(B5_TV2_cracks)
dic_insp.interact()

c:\users\fahad\pycharmprojects\bmcs_shear_zone\bmcs_shear\dic_crack\dic_cor.py:142: RuntimeWarning: invalid value encountered in true_divide
  phi = 2 * np.arctan(len_d_0t / len_d_0c)
c:\users\fahad\pycharmprojects\bmcs_shear_zone\bmcs_shear\dic_crack\dic_cor.py:142: RuntimeWarning: invalid value encountered in true_divide
  phi = 2 * np.arctan(len_d_0t / len_d_0c)
c:\users\fahad\pycharmprojects\bmcs_shear_zone\bmcs_shear\dic_crack\dic_cor.py:142: RuntimeWarning: invalid value encountered in true_divide
  phi = 2 * np.arctan(len_d_0t / len_d_0c)


# B6_TV1

In [17]:
B6_TV1_cracks = [
    # crask 1
    (dict(y_ref_i=-1, y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min = -10, n_x_max = -9, n_x_step=1, n_y_min = 1, n_y_max=10, n_y_step = 1)),
    # crack 2
    (dict(y_ref_i=-10, y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min = -16, n_x_max = -13, n_x_step=1, n_y_min = 1, n_y_max=10, n_y_step = 1)),
    # crack 3
    (dict(y_ref_i=-15, y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min = -20, n_x_max = -18, n_x_step=1, n_y_min = 1, n_y_max=7, n_y_step = 1)),
    # crack 4
    (dict(y_ref_i=-21, y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min = -29, n_x_max = -23, n_x_step=1, n_y_min = 4, n_y_max=-1, n_y_step = 1)),
    # crack 5
    (dict(y_ref_i=31, y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min = 21, n_x_max = 26, n_x_step=1, n_y_min =3, n_y_max=-1, n_y_step = 1)),
    # crack 6
    (dict(y_ref_i=22, y_ref_j_min=1, y_ref_j_max=10),#22
     dict(n_x_min = 1, n_x_max = 14, n_x_step=2, n_y_min = 3, n_y_max=-3, n_y_step = 2))
]

In [18]:
dic_grid = DICGrid(n_x=58, n_y=15, d_x = 22, d_y = 22, start_t = 0, end_t =31, U_factor = 1, 
                   dir_name='B6_TV1', grid_column_first = False, grid_number_vertical = False)
dic_insp = DICInspector(dic_grid=dic_grid)
dic_insp.dic_cracks.items = construct_dic_cracks(B6_TV1_cracks)
dic_insp.interact()

c:\users\fahad\pycharmprojects\bmcs_shear_zone\bmcs_shear\dic_crack\dic_cor.py:142: RuntimeWarning: invalid value encountered in true_divide
  phi = 2 * np.arctan(len_d_0t / len_d_0c)
c:\users\fahad\pycharmprojects\bmcs_shear_zone\bmcs_shear\dic_crack\dic_cor.py:142: RuntimeWarning: invalid value encountered in true_divide
  phi = 2 * np.arctan(len_d_0t / len_d_0c)
c:\users\fahad\pycharmprojects\bmcs_shear_zone\bmcs_shear\dic_crack\dic_cor.py:142: RuntimeWarning: invalid value encountered in true_divide
  phi = 2 * np.arctan(len_d_0t / len_d_0c)
c:\users\fahad\pycharmprojects\bmcs_shear_zone\bmcs_shear\dic_crack\dic_cor.py:142: RuntimeWarning: invalid value encountered in true_divide
  phi = 2 * np.arctan(len_d_0t / len_d_0c)


# B6_TV2

In [19]:
B6_TV2_cracks = [
    # crask 1
    (dict(y_ref_i=-1, y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min = -7, n_x_max = -4, n_x_step=1, n_y_min = 1, n_y_max=10, n_y_step = 1)),
    # crack 2
    (dict(y_ref_i=-7, y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min = -15, n_x_max = -12, n_x_step=1, n_y_min = 5, n_y_max=12, n_y_step = 1)),
    # crack 3
    (dict(y_ref_i=-15, y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min = -25, n_x_max = -21, n_x_step=1, n_y_min = 7, n_y_max=12, n_y_step = 1)),
    # crack 4
    (dict(y_ref_i=20, y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min = 1, n_x_max = 14, n_x_step=1, n_y_min = 1, n_y_max=-1, n_y_step = 1)),
#     # crack 5
#     (dict(y_ref_i=31, y_ref_j_min=1, y_ref_j_max=10),
#      dict(n_x_min = 21, n_x_max = 26, n_x_step=1, n_y_min =3, n_y_max=-1, n_y_step = 1)),
#     # crack 6
#     (dict(y_ref_i=22, y_ref_j_min=1, y_ref_j_max=10),#22
#      dict(n_x_min = 1, n_x_max = 14, n_x_step=2, n_y_min = 3, n_y_max=-3, n_y_step = 2))
]

In [20]:
dic_grid = DICGrid(n_x=57, n_y=13, d_x = 22, d_y = 22, start_t = 0, end_t =19, U_factor = 1, 
                   dir_name='B6_TV2_', grid_column_first = False, grid_number_vertical = False)
dic_insp = DICInspector(dic_grid=dic_grid)
dic_insp.dic_cracks.items = construct_dic_cracks(B6_TV2_cracks)
dic_insp.interact()

c:\users\fahad\pycharmprojects\bmcs_shear_zone\bmcs_shear\dic_crack\dic_cor.py:142: RuntimeWarning: invalid value encountered in true_divide
  phi = 2 * np.arctan(len_d_0t / len_d_0c)
c:\users\fahad\pycharmprojects\bmcs_shear_zone\bmcs_shear\dic_crack\dic_cor.py:142: RuntimeWarning: invalid value encountered in true_divide
  phi = 2 * np.arctan(len_d_0t / len_d_0c)
c:\users\fahad\pycharmprojects\bmcs_shear_zone\bmcs_shear\dic_crack\dic_cor.py:142: RuntimeWarning: invalid value encountered in true_divide
  phi = 2 * np.arctan(len_d_0t / len_d_0c)


# B7_TV1

In [23]:
B7_TV1_cracks = [
    # crask 1
    (dict(y_ref_i=-1, y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min = -12, n_x_max = -8, n_x_step=1, n_y_min = 1, n_y_max=10, n_y_step = 1)),
    # crack 2
    (dict(y_ref_i=-12, y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min = -19, n_x_max = -16, n_x_step=1, n_y_min = 1, n_y_max=10, n_y_step = 1)),
     # crack 3
     (dict(y_ref_i=-19, y_ref_j_min=1, y_ref_j_max=10),
      dict(n_x_min = 32, n_x_max = 35, n_x_step=1, n_y_min = 1, n_y_max=8, n_y_step = 1)),
     # crack 4
     (dict(y_ref_i=32, y_ref_j_min=1, y_ref_j_max=10),
      dict(n_x_min = 23, n_x_max = 26, n_x_step=1, n_y_min = 1, n_y_max=7, n_y_step = 1)),
     # crack 5
     (dict(y_ref_i=25, y_ref_j_min=1, y_ref_j_max=10),
      dict(n_x_min = 0, n_x_max = 18, n_x_step=2, n_y_min = 1, n_y_max=-1, n_y_step = 2)),
]

In [24]:
dic_grid = DICGrid(n_x=58, n_y=14, d_x = 22, d_y = 22, start_t = 0, end_t =25, U_factor = 1, 
                   dir_name='B7_TV1', grid_column_first = False, grid_number_vertical = False)
dic_insp = DICInspector(dic_grid=dic_grid)
dic_insp.dic_cracks.items = construct_dic_cracks(B7_TV1_cracks)
dic_insp.interact()

evaluating step 0
phi_avg 0.0
evaluating step 1
phi_avg 0.00011141290386465849
evaluating step 2
phi_avg 0.00013300995536972714
evaluating step 3
phi_avg 0.00031612524734117066
evaluating step 4
phi_avg 0.0006827830277979685
evaluating step 5
phi_avg 0.000840919012208924
evaluating step 6
phi_avg 0.001135403197348864
evaluating step 7
phi_avg 0.0013288220137698146
evaluating step 8
phi_avg 0.0017017972114850718
evaluating step 9
phi_avg 0.002027776671251026
evaluating step 10
phi_avg 0.0022778952881165257
evaluating step 11
phi_avg 0.0023261105625896716
evaluating step 12
phi_avg 0.002296081551005789
evaluating step 13
phi_avg 0.0024649415208334085
evaluating step 14
phi_avg 0.0026307182331956527
evaluating step 15
phi_avg 0.002510754590645288
evaluating step 16
phi_avg 0.0025554033734338883
evaluating step 17
phi_avg 0.0025427261774971788
evaluating step 18
phi_avg 0.00257567559180902
evaluating step 19
phi_avg 0.002586592076983337
evaluating step 20
phi_avg 0.002657674268942046
evalu

c:\users\fahad\pycharmprojects\bmcs_shear_zone\bmcs_shear\dic_crack\dic_cor.py:142: RuntimeWarning: invalid value encountered in true_divide
  phi = 2 * np.arctan(len_d_0t / len_d_0c)


phi_avg 0.0
evaluating step 1
phi_avg 0.00011587510509033528
evaluating step 2
phi_avg 8.848986612788287e-05
evaluating step 3
phi_avg 0.00020686768670395207
evaluating step 4
phi_avg 0.0006280137790971821
evaluating step 5
phi_avg 0.0007080061968819317
evaluating step 6
phi_avg 0.0008065296294776015
evaluating step 7
phi_avg 0.000881785934560829
evaluating step 8
phi_avg 0.0011117447749438417
evaluating step 9
phi_avg 0.001342898979897236
evaluating step 10
phi_avg 0.0014645086384244427
evaluating step 11
phi_avg 0.0014883635500573089
evaluating step 12
phi_avg 0.001503472182119669
evaluating step 13
phi_avg 0.0014994796605752756
evaluating step 14
phi_avg 0.0015335313448746539
evaluating step 15
phi_avg 0.0016924517384684509
evaluating step 16
phi_avg 0.001574411596030668
evaluating step 17
phi_avg 0.0015194321960721872
evaluating step 18
phi_avg 0.0015867783923134951
evaluating step 19
phi_avg 0.0015838045148726945
evaluating step 20
phi_avg 0.001559316147902921
evaluating step 21
p

c:\users\fahad\pycharmprojects\bmcs_shear_zone\bmcs_shear\dic_crack\dic_cor.py:142: RuntimeWarning: invalid value encountered in true_divide
  phi = 2 * np.arctan(len_d_0t / len_d_0c)


phi_avg 0.0
evaluating step 1
phi_avg 0.00015283495182353207
evaluating step 2
phi_avg 0.00018970818372580691
evaluating step 3
phi_avg 0.0002565710055545414
evaluating step 4
phi_avg 0.00028961596209030625
evaluating step 5
phi_avg 0.0002860997764840788
evaluating step 6
phi_avg 0.000514154139508621
evaluating step 7
phi_avg 0.0006617186838175688
evaluating step 8
phi_avg 0.0007434547513625881
evaluating step 9
phi_avg 0.0008234580078907862
evaluating step 10
phi_avg 0.0010175493491134473
evaluating step 11
phi_avg 0.0008838258550682933
evaluating step 12
phi_avg 0.0008881810192438172
evaluating step 13
phi_avg 0.000981152775808253
evaluating step 14
phi_avg 0.0009796520918866531
evaluating step 15
phi_avg 0.0011356289267335804
evaluating step 16
phi_avg 0.0010349164683505195
evaluating step 17
phi_avg 0.0009662432024815223
evaluating step 18
phi_avg 0.0009695381086518516
evaluating step 19
phi_avg 0.0009925032725041916
evaluating step 20
phi_avg 0.0009008124071562355
evaluating step 

# B7_TV2

In [25]:
B7_TV2_cracks = [
    # crask 1
    (dict(y_ref_i=-1, y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min = -8, n_x_max = -6, n_x_step=1, n_y_min = 1, n_y_max=10, n_y_step = 1)),
    # crack 2
    (dict(y_ref_i=-8, y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min = -13, n_x_max = -11, n_x_step=1, n_y_min = 1, n_y_max=10, n_y_step = 1)),
     # crack 3
     (dict(y_ref_i=-13, y_ref_j_min=1, y_ref_j_max=10),
      dict(n_x_min = -23, n_x_max = -18, n_x_step=1, n_y_min = 3, n_y_max=8, n_y_step = 1)),
     # crack 4
     (dict(y_ref_i=-23, y_ref_j_min=1, y_ref_j_max=10),
      dict(n_x_min = -28, n_x_max = -24, n_x_step=1, n_y_min = 3, n_y_max=7, n_y_step = 1)),
     # crack 5
     (dict(y_ref_i=-28, y_ref_j_min=1, y_ref_j_max=10),
      dict(n_x_min = 1, n_x_max = 12, n_x_step=1, n_y_min = 1, n_y_max=-1, n_y_step = 1)),
]

In [26]:
dic_grid = DICGrid(n_x=53, n_y=12, d_x = 23, d_y = 23, start_t = 0, end_t =15, U_factor = 1, 
                   dir_name='B7_TV2', grid_column_first = True, grid_number_vertical = True)
dic_insp = DICInspector(dic_grid=dic_grid)
dic_insp.dic_cracks.items = construct_dic_cracks(B7_TV2_cracks)
dic_insp.interact()

evaluating step 0
phi_avg 0.0
evaluating step 1
phi_avg 0.00018042559987524551
evaluating step 2
phi_avg 0.0002006973464519511
evaluating step 3
phi_avg 0.00011407478474649171
evaluating step 4
phi_avg 0.00010559218971922248
evaluating step 5
phi_avg 0.00036120955289678454
evaluating step 6
phi_avg 0.0005336206645646991
evaluating step 7
phi_avg 0.0007182781011176551
evaluating step 8
phi_avg 0.0008831663973533568
evaluating step 9
phi_avg 0.0010084803755225523
evaluating step 10
phi_avg 0.001025466726246144
evaluating step 11
phi_avg 0.001161754797790918
evaluating step 12
phi_avg 0.0013605320952935448
evaluating step 13
phi_avg 0.0014071067026064047
evaluating step 14
phi_avg 0.0015564390418429665
evaluating step 0
phi_avg 0.0
evaluating step 1
phi_avg 0.00015040009373235724
evaluating step 2
phi_avg 9.169834643183831e-05
evaluating step 3
phi_avg 0.00020726281093839643
evaluating step 4
phi_avg 0.0005560691708006393
evaluating step 5
phi_avg 0.0006330785514163969
evaluating step 6
p

c:\users\fahad\pycharmprojects\bmcs_shear_zone\bmcs_shear\dic_crack\dic_cor.py:142: RuntimeWarning: invalid value encountered in true_divide
  phi = 2 * np.arctan(len_d_0t / len_d_0c)


phi_avg 0.0002608998922774604
evaluating step 2
phi_avg 0.0002453306749989491
evaluating step 3
phi_avg 0.00020944282867168325
evaluating step 4
phi_avg 0.00019458185413108908
evaluating step 5
phi_avg 0.0006037539985879925
evaluating step 6
phi_avg 0.0007268882011423872
evaluating step 7
phi_avg 0.0008743145084408679
evaluating step 8
phi_avg 0.0010232877811425102
evaluating step 9
phi_avg 0.0012381058499866093
evaluating step 10
phi_avg 0.0012061041693446795
evaluating step 11
phi_avg 0.0014070415351407353
evaluating step 12
phi_avg 0.0017126518669092628
evaluating step 0
phi_avg 0.0
evaluating step 1
phi_avg 0.00015003173242170633
evaluating step 2
phi_avg 0.000510209867731805
evaluating step 3
phi_avg 0.00032149640548381726
evaluating step 4
phi_avg 0.0003950565419318289
evaluating step 5
phi_avg 0.0005058706852904002
evaluating step 6
phi_avg 0.0005165884609153363
evaluating step 7
phi_avg 0.000697567982766163
evaluating step 8
phi_avg 0.000799427670429075
evaluating step 9
phi_av

# B8_TV1

In [27]:
B8_TV1_cracks = [
    # crack 1
    (dict(y_ref_i=-1 , y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min = -15, n_x_max = -8, n_x_step=1, n_y_min = 5, n_y_max=-2, n_y_step = 1)),
    # crack 2
    (dict(y_ref_i=-13, y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min = -19, n_x_max = -17, n_x_step=1, n_y_min = 8, n_y_max=-1, n_y_step = 1)),
    # crack 3
    (dict(y_ref_i=-19, y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min = -24, n_x_max = -21, n_x_step=1, n_y_min = 5, n_y_max=-1, n_y_step = 1)),
    # crack 4
    (dict(y_ref_i=-24, y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min = -33, n_x_max = -29, n_x_step=1, n_y_min = 7, n_y_step = 1)),
    # crack 5
    (dict(y_ref_i=-33, y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min=1, n_x_max=15, n_x_step=1, n_y_min = 2, n_y_max=12, n_y_step = 1)),
]

In [28]:
dic_grid = DICGrid(n_x=56, n_y=13, d_x = 23, d_y = 23,  start_t = 0, end_t =14, U_factor = 1, 
                   dir_name='B8_TV1', grid_column_first = True, grid_number_vertical = True)
dic_insp = DICInspector(dic_grid=dic_grid)
dic_insp.dic_cracks.items = construct_dic_cracks(B8_TV1_cracks)
dic_insp.interact()

evaluating step 0
phi_avg 0.0
evaluating step 1
phi_avg 9.48314617354356e-05
evaluating step 2
phi_avg 0.00041579210623353847
evaluating step 3
phi_avg 0.00014594776082000512
evaluating step 4
phi_avg 0.0002830194702041944
evaluating step 5
phi_avg 0.0004094221929633157
evaluating step 6
phi_avg 0.0005983097942808978
evaluating step 7
phi_avg 0.000776678126959806
evaluating step 8
phi_avg 0.0009739994621828973
evaluating step 9
phi_avg 0.0011324248864598284
evaluating step 10
phi_avg 0.0013929463924097977
evaluating step 11
phi_avg 0.0015451241994845597
evaluating step 12
phi_avg 0.0018105465053917694
evaluating step 13
phi_avg 0.00192704209101195
evaluating step 0
phi_avg 0.0
evaluating step 1
phi_avg 0.00024447246478967596
evaluating step 2
phi_avg 0.00016824094065373313
evaluating step 3
phi_avg 0.0002747286779520757
evaluating step 4
phi_avg 0.0003277452425221258
evaluating step 5
phi_avg 0.0004797462227602203
evaluating step 6
phi_avg 0.0005940739512724958
evaluating step 7
phi_av

c:\users\fahad\pycharmprojects\bmcs_shear_zone\bmcs_shear\dic_crack\dic_cor.py:142: RuntimeWarning: invalid value encountered in true_divide
  phi = 2 * np.arctan(len_d_0t / len_d_0c)


phi_avg 0.00018123393430352015
evaluating step 2
phi_avg 0.00017752694372637874
evaluating step 3
phi_avg 0.00010936429448178393
evaluating step 4
phi_avg 0.0001365537661055326
evaluating step 5
phi_avg 0.00013339678439360552
evaluating step 6
phi_avg 0.00019962759769374722
evaluating step 7
phi_avg 0.00022881926030111327
evaluating step 8
phi_avg 0.00016412797516856097
evaluating step 9
phi_avg 0.0002603865782997551
evaluating step 10
phi_avg 0.0002625819512891287
evaluating step 11
phi_avg 0.00035646752101581167
evaluating step 0
phi_avg 0.0
evaluating step 1
phi_avg 6.204549836295909e-05
evaluating step 2
phi_avg 0.00010189434710348483
evaluating step 3
phi_avg 0.0001905314982856403
evaluating step 4
phi_avg 0.00024254483955690395
evaluating step 5
phi_avg 0.0003068560962312404
evaluating step 6
phi_avg 0.0004457034221097528
evaluating step 7
phi_avg 0.0005579856682457753
evaluating step 8
phi_avg 0.0008003319141151967
evaluating step 9
phi_avg 0.0008617601664512043
evaluating step 

# B8_TV2

In [29]:
B8_TV2_cracks = [
    # crack 1
    (dict(y_ref_i=-1 , y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min = -8, n_x_max = -3, n_x_step=1, n_y_min = 5, n_y_max=-2, n_y_step = 1)),
    # crack 2
    (dict(y_ref_i=-8, y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min = -16, n_x_max = -13, n_x_step=1, n_y_min = 5, n_y_max=11, n_y_step = 1)),
    # crack 3
    (dict(y_ref_i=-16, y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min = -25, n_x_max = -21, n_x_step=1, n_y_min = 5, n_y_max=10, n_y_step = 1)),
    # crack 4
    (dict(y_ref_i=-24, y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min = -35, n_x_max = -31, n_x_step=1, n_y_min = 5, n_y_step = 1)),
    # crack 5
    (dict(y_ref_i=-33, y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min=1, n_x_max=14, n_x_step=1, n_y_min = 2, n_y_max=12, n_y_step = 1)),
]

In [30]:
dic_grid = DICGrid(n_x=56, n_y=14, d_x = 23, d_y = 23,  start_t = 0, end_t =15, U_factor = 1, 
                   dir_name='B8_TV2', grid_column_first = True, grid_number_vertical = True)
dic_insp = DICInspector(dic_grid=dic_grid)
dic_insp.dic_cracks.items = construct_dic_cracks(B8_TV2_cracks)
dic_insp.interact()

evaluating step 0
phi_avg 0.0
evaluating step 1


c:\users\fahad\pycharmprojects\bmcs_shear_zone\bmcs_shear\dic_crack\dic_cor.py:142: RuntimeWarning: invalid value encountered in true_divide
  phi = 2 * np.arctan(len_d_0t / len_d_0c)


phi_avg 9.060903285157222e-05
evaluating step 2
phi_avg 0.00022491418292242113
evaluating step 3
phi_avg 0.0004068793077469551
evaluating step 4
phi_avg 0.000626429754043621
evaluating step 5
phi_avg 0.0008256085398137159
evaluating step 6
phi_avg 0.001121485566110636
evaluating step 7
phi_avg 0.0013150163386231994
evaluating step 8
phi_avg 0.0016217254098150338
evaluating step 9
phi_avg 0.001811776800017737
evaluating step 10
phi_avg 0.002006305385211729
evaluating step 11
phi_avg 0.0022559860230635365
evaluating step 12
phi_avg 0.0024966186347272325
evaluating step 13
phi_avg 0.002702144792645716
evaluating step 14
phi_avg 0.0030301431040953425
evaluating step 0
phi_avg 0.0
evaluating step 1
phi_avg 8.772263721893118e-05
evaluating step 2
phi_avg 0.00011976955833883258
evaluating step 3
phi_avg 0.00024545684955257604
evaluating step 4
phi_avg 0.0004735380010241613
evaluating step 5
phi_avg 0.0006348235365345435
evaluating step 6
phi_avg 0.000831424943969695
evaluating step 7
phi_avg 

# B9_TV1

In [31]:
B9_TV1_cracks = [
    # crack 1
    (dict(y_ref_i=-1 , y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min = -9, n_x_max = -6, n_x_step=1, n_y_min = 5, n_y_max=-2, n_y_step = 1)),
    # crack 2
    (dict(y_ref_i=-9, y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min = -15, n_x_max = -12, n_x_step=1, n_y_min = 1, n_y_max=4, n_y_step = 1)),
    # crack 3
    (dict(y_ref_i=-14, y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min = -19, n_x_max = -16, n_x_step=1, n_y_min = 5, n_y_max=10, n_y_step = 1)),
    # crack 4
    (dict(y_ref_i=-19, y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min = -25, n_x_max = -22, n_x_step=1, n_y_min = 3, n_y_max = 8, n_y_step = 1)),
    # crack 5
    (dict(y_ref_i=-25, y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min=-30, n_x_max=-28, n_x_step=1, n_y_min = 3, n_y_max=8, n_y_step = 1)),
    # crack 6
    (dict(y_ref_i=-30, y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min=1, n_x_max=14, n_x_step=1, n_y_min = 1, n_y_max=10, n_y_step = 1)),
]

In [32]:
dic_grid = DICGrid(n_x=54, n_y=12, d_x = 23, d_y = 23, start_t = 0, end_t =18, U_factor = 1, 
                   dir_name='B9_TV1_', grid_column_first = True, grid_number_vertical = True)
dic_insp = DICInspector(dic_grid=dic_grid)
dic_insp.dic_cracks.items = construct_dic_cracks(B9_TV1_cracks)
dic_insp.interact()

evaluating step 0
phi_avg 0.0
evaluating step 1
phi_avg 0.00016216638311275421
evaluating step 2


c:\users\fahad\pycharmprojects\bmcs_shear_zone\bmcs_shear\dic_crack\dic_cor.py:142: RuntimeWarning: invalid value encountered in true_divide
  phi = 2 * np.arctan(len_d_0t / len_d_0c)


phi_avg 0.00014918939554631132
evaluating step 3
phi_avg 0.00031639436922208336
evaluating step 4
phi_avg 0.00030859886147312476
evaluating step 5
phi_avg 0.0003297485765047232
evaluating step 6
phi_avg 0.0005566507593045314
evaluating step 7
phi_avg 0.0006123749494793821
evaluating step 8
phi_avg 0.00078011716543488
evaluating step 9
phi_avg 0.0009763494410712457
evaluating step 10
phi_avg 0.0011090362486101324
evaluating step 11
phi_avg 0.0013138334192122289
evaluating step 12
phi_avg 0.0014088125307509176
evaluating step 13
phi_avg 0.0014122793489073668
evaluating step 14
phi_avg 0.0015800675864847747
evaluating step 15
phi_avg 0.0017522369840164752
evaluating step 16
phi_avg 0.0017492685628319605
evaluating step 17
phi_avg 0.0018089694157065574
evaluating step 0
phi_avg 0.0
evaluating step 1


c:\users\fahad\pycharmprojects\bmcs_shear_zone\bmcs_shear\dic_crack\dic_cor.py:142: RuntimeWarning: invalid value encountered in true_divide
  phi = 2 * np.arctan(len_d_0t / len_d_0c)


phi_avg 0.00029288935154208424
evaluating step 2
phi_avg 0.0003789926516589577
evaluating step 3
phi_avg 0.000294442231458687
evaluating step 4
phi_avg 0.0005048539162507564
evaluating step 5
phi_avg 0.0005823241442785564
evaluating step 6
phi_avg 0.001190147086453719
evaluating step 7
phi_avg 0.0036288031333709617
evaluating step 8
phi_avg 0.0019002885719604126
evaluating step 9
phi_avg 0.001789140602351612
evaluating step 10
phi_avg 0.0018619582557064425
evaluating step 11
phi_avg 0.0014511085558491811
evaluating step 12
phi_avg 0.0015117931057595825
evaluating step 13
phi_avg 0.0014081187776383436
evaluating step 14
phi_avg 0.0014974965051819857
evaluating step 15
phi_avg 0.0017284992656743216
evaluating step 16
phi_avg 0.0017535396902148435
evaluating step 0


...

phi_avg 0.0
evaluating step 1
phi_avg 0.00021074535870570793
evaluating step 2
phi_avg 0.00025780318564262364
evaluating step 3
phi_avg 0.0003772249556338764
evaluating step 4
phi_avg 0.00030292629542671393
evaluating step 5
phi_avg 0.00014194665738820298
evaluating step 6
phi_avg 0.0005701662947808064
evaluating step 7
phi_avg 0.0009335816027720739
evaluating step 8
phi_avg 0.0011315927210188202
evaluating step 9
phi_avg 0.0013699589492023087
evaluating step 10
phi_avg 0.0016063700079885855
evaluating step 11
phi_avg 0.0015082128818886124
evaluating step 12
phi_avg 0.0016509441146574306
evaluating step 13
phi_avg 0.001652657590783617
evaluating step 14
phi_avg 0.001897730908523382
evaluating step 15
phi_avg 0.002177722814039781
evaluating step 0


c:\users\fahad\pycharmprojects\bmcs_shear_zone\bmcs_shear\dic_crack\dic_cor.py:142: RuntimeWarning: invalid value encountered in true_divide
  phi = 2 * np.arctan(len_d_0t / len_d_0c)


phi_avg 0.0
evaluating step 1
phi_avg 0.0003092486501733007
evaluating step 2
phi_avg 0.00024241610540695708
evaluating step 3
phi_avg 0.00024144546907140435
evaluating step 4
phi_avg 0.0008091169067992338
evaluating step 5
phi_avg 0.00028689628461073794
evaluating step 6
phi_avg 0.00046726130824211424
evaluating step 7
phi_avg 0.0003558689441226781
evaluating step 8
phi_avg 0.000473695696857216
evaluating step 9
phi_avg 0.0006093219791126345
evaluating step 10
phi_avg 0.0007936223870743294
evaluating step 11
phi_avg 0.001006897614980728
evaluating step 12
phi_avg 0.0010716882029514134
evaluating step 13
phi_avg 0.0010586477061242593
evaluating step 14
phi_avg 0.0010837843852314608
evaluating step 0
phi_avg 0.0
evaluating step 1
phi_avg 0.0007913423159449902
evaluating step 2
phi_avg 0.00024919181705295677
evaluating step 3
phi_avg 0.0003554510422271217
evaluating step 4
phi_avg 0.000647122256073366
evaluating step 5
phi_avg 0.00035597522024318043
evaluating step 6
phi_avg 0.0003678016

c:\users\fahad\pycharmprojects\bmcs_shear_zone\bmcs_shear\dic_crack\dic_cor.py:142: RuntimeWarning: invalid value encountered in true_divide
  phi = 2 * np.arctan(len_d_0t / len_d_0c)


phi_avg 0.00025417163612329903
evaluating step 2
phi_avg 0.00021733822341593573
evaluating step 3
phi_avg 0.00022534177559669393
evaluating step 4
phi_avg 0.0002061534332301122
evaluating step 5
phi_avg 0.0002398629203474157
evaluating step 6
phi_avg 0.0002300492842038118
evaluating step 7
phi_avg 0.0002466458182705637
evaluating step 8
phi_avg 0.0002489789816939101
evaluating step 9
phi_avg 0.000295835091511878
evaluating step 10
phi_avg 0.0004558395240008791
evaluating step 11
phi_avg 0.0006306968961522453
evaluating step 12
phi_avg 0.0010788277231452178


# B9_TV2

In [33]:
B9_TV2_cracks = [
    # crack 1
    (dict(y_ref_i=-1 , y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min = -9, n_x_max = -6, n_x_step=1, n_y_min = 5, n_y_max=-2, n_y_step = 1)),
    # crack 2
    (dict(y_ref_i=-9, y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min = -15, n_x_max = -11, n_x_step=1, n_y_min = 6, n_y_max=10, n_y_step = 1)),
    # crack 3
    (dict(y_ref_i=-14, y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min = -20, n_x_max = -17, n_x_step=1, n_y_min = 5, n_y_max=10, n_y_step = 1)),
    # crack 4
    (dict(y_ref_i=-19, y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min = -25, n_x_max = -22, n_x_step=1, n_y_min = 3, n_y_max = 8, n_y_step = 1)),
    # crack 5
    (dict(y_ref_i=-25, y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min=-33, n_x_max=-30, n_x_step=1, n_y_min = 6, n_y_max=10, n_y_step = 1)),
    # crack 6
    (dict(y_ref_i=-33, y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min=1, n_x_max=14, n_x_step=1, n_y_min = 1, n_y_max=10, n_y_step = 1)),
]

In [34]:
dic_grid = DICGrid(n_x=57, n_y=13, d_x = 22, d_y = 22, start_t = 0, end_t =16, U_factor = 1, 
                   dir_name='B9_TV2', grid_column_first = True, grid_number_vertical = True)
dic_insp = DICInspector(dic_grid=dic_grid)
dic_insp.dic_cracks.items = construct_dic_cracks(B9_TV2_cracks)
dic_insp.interact()

evaluating step 0
phi_avg 0.0
evaluating step 1
phi_avg 9.952042681228787e-05
evaluating step 2
phi_avg 0.00010548302991555993
evaluating step 3
phi_avg 0.0002654261577644264
evaluating step 4
phi_avg 0.00034318630228252994
evaluating step 5
phi_avg 0.000565376785457412
evaluating step 6
phi_avg 0.0007208055553099701
evaluating step 7
phi_avg 0.0009184350472513826
evaluating step 8
phi_avg 0.0013095010729629241
evaluating step 9
phi_avg 0.001439383795610855
evaluating step 10
phi_avg 0.001820283713683343
evaluating step 11
phi_avg 0.002068870818415064
evaluating step 12
phi_avg 0.0023520923048134437
evaluating step 13
phi_avg 0.0026096868019360574
evaluating step 14
phi_avg 0.0028707724947472564
evaluating step 15
phi_avg 0.0031053762037716167
evaluating step 0
phi_avg 0.0
evaluating step 1
phi_avg 0.00020841245725877983
evaluating step 2
phi_avg 0.00017243854522010893
evaluating step 3
phi_avg 0.00020352616943178825
evaluating step 4
phi_avg 0.0002426894402646211
evaluating step 5
phi

c:\users\fahad\pycharmprojects\bmcs_shear_zone\bmcs_shear\dic_crack\dic_cor.py:142: RuntimeWarning: invalid value encountered in true_divide
  phi = 2 * np.arctan(len_d_0t / len_d_0c)


phi_avg 9.251483020780558e-05
evaluating step 2
phi_avg 0.0002978006444295784
evaluating step 3
phi_avg 0.00022093350387546659
evaluating step 4
phi_avg 0.00019233983580544512
evaluating step 5
phi_avg 0.000299074708695254
evaluating step 6
phi_avg 0.0004332676192565511
evaluating step 7
phi_avg 0.0005530213506973315
evaluating step 8
phi_avg 0.0008009583149067248
evaluating step 9
phi_avg 0.0007740528736340321
evaluating step 10
phi_avg 0.001065253357756332
evaluating step 11
phi_avg 0.0009703571483328193
evaluating step 12
phi_avg 0.0010966448453647462
evaluating step 13
phi_avg 0.0012809965556911797
evaluating step 0
phi_avg 0.0
evaluating step 1


c:\users\fahad\pycharmprojects\bmcs_shear_zone\bmcs_shear\dic_crack\dic_cor.py:142: RuntimeWarning: invalid value encountered in true_divide
  phi = 2 * np.arctan(len_d_0t / len_d_0c)


phi_avg 0.00014199877700448924
evaluating step 2
phi_avg 0.00026778757681469704
evaluating step 3
phi_avg 0.00038123464051415304
evaluating step 4
phi_avg 0.00012055016820895743
evaluating step 5
phi_avg 0.00024973526353475826
evaluating step 6
phi_avg 0.00010787151295417905
evaluating step 7
phi_avg 0.0002319624406433748
evaluating step 8
phi_avg 0.00032816409317647726
evaluating step 9
phi_avg 0.0005172896038229374
evaluating step 10
phi_avg 0.0007821526484797297
evaluating step 11
phi_avg 0.000595526169338261
evaluating step 12
phi_avg 0.0007266151980956614
evaluating step 0
phi_avg 0.0
evaluating step 1
phi_avg 0.00015840334069866405
evaluating step 2
phi_avg 0.00028902214145036515
evaluating step 3
phi_avg 0.000186335637565155
evaluating step 4
phi_avg 0.00012057336948512904
evaluating step 5
phi_avg 0.00020183817669434335
evaluating step 6
phi_avg 0.0005693765078767193
evaluating step 7
phi_avg 0.0003251409417868634
evaluating step 8
phi_avg 0.00039310757038449237
evaluating step

c:\users\fahad\pycharmprojects\bmcs_shear_zone\bmcs_shear\dic_crack\dic_cor.py:142: RuntimeWarning: invalid value encountered in true_divide
  phi = 2 * np.arctan(len_d_0t / len_d_0c)


phi_avg 0.00014273407002579337
evaluating step 2
phi_avg 0.0001569264978017126
evaluating step 3
phi_avg 0.00011461129465088612
evaluating step 4
phi_avg 0.00017796071716141653
evaluating step 5
phi_avg 0.00019476668825619273
evaluating step 6
phi_avg 0.00021236482943164252
evaluating step 7
phi_avg 0.0002410317810874941
evaluating step 8
phi_avg 0.00016059690455963013
evaluating step 9
phi_avg 0.0002467109848205844
evaluating step 10
phi_avg 0.00034544272239252764


# B10_TV1

In [35]:
B10_TV1_cracks = [
    # crack 1
    (dict(y_ref_i=-1 , y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min = -6, n_x_max = -3, n_x_step=1, n_y_min = 1, n_y_max=5, n_y_step = 1)),
    # crack 2
    (dict(y_ref_i=-6, y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min = -16, n_x_max = -13, n_x_step=1, n_y_min = 6, n_y_max=10, n_y_step = 1)),
    # crack 3
    (dict(y_ref_i=-14, y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min = -25, n_x_max = -22, n_x_step=1, n_y_min = 5, n_y_max=10, n_y_step = 1)),
    # crack 4
    (dict(y_ref_i=-25, y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min = -33, n_x_max = -28, n_x_step=1, n_y_min = 7, n_y_max = 12, n_y_step = 1)),
    # crack 5
    (dict(y_ref_i=-33, y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min=1, n_x_max=14, n_x_step=1, n_y_min = 1, n_y_max=12, n_y_step = 1)),
]

In [36]:
dic_grid = DICGrid(n_x=57, n_y=14, d_x = 22, d_y = 22, start_t = 0, end_t =15, U_factor = 1, 
                   dir_name='B10_TV1', grid_column_first = True, grid_number_vertical = True)
dic_insp = DICInspector(dic_grid=dic_grid)
dic_insp.dic_cracks.items = construct_dic_cracks(B10_TV1_cracks)
dic_insp.interact()

evaluating step 0
phi_avg 0.0
evaluating step 1
phi_avg 0.00019145905435867693
evaluating step 2
phi_avg 0.001254696176576462
evaluating step 3
phi_avg 0.0024884723624373005
evaluating step 4
phi_avg 0.002313235260341996
evaluating step 5
phi_avg 0.0024865227965158235
evaluating step 6
phi_avg 0.0030493904474457084
evaluating step 7
phi_avg 0.004200067502043321
evaluating step 8
phi_avg 0.0025715661649576347
evaluating step 9
phi_avg 0.004187289318433293
evaluating step 10
phi_avg 0.0038942766947831727
evaluating step 11
phi_avg 0.003948274892105988
evaluating step 12
phi_avg 0.004135323244170747
evaluating step 13
phi_avg 0.0035030887150102833
evaluating step 14
phi_avg 0.0037720022479588258
evaluating step 0
phi_avg 0.0
evaluating step 1
phi_avg 0.00013261549297445663
evaluating step 2
phi_avg 0.000176920330946592
evaluating step 3
phi_avg 0.0005252468068019927
evaluating step 4
phi_avg 0.0009964238482659796
evaluating step 5
phi_avg 0.0013340188092912841
evaluating step 6
phi_avg 0.

c:\users\fahad\pycharmprojects\bmcs_shear_zone\bmcs_shear\dic_crack\dic_cor.py:142: RuntimeWarning: invalid value encountered in true_divide
  phi = 2 * np.arctan(len_d_0t / len_d_0c)


phi_avg 0.00028827825761778387
evaluating step 2
phi_avg 0.00018993593112479732
evaluating step 3
phi_avg 0.00035564192715344903
evaluating step 4
phi_avg 0.00040700203795196303
evaluating step 5
phi_avg 0.0005585599331875909
evaluating step 6
phi_avg 0.00074722612485147
evaluating step 7
phi_avg 0.0008442502787787062
evaluating step 8
phi_avg 0.001009668048618919
evaluating step 9
phi_avg 0.0009172093603642715
evaluating step 10
phi_avg 0.0010841406382118698
evaluating step 11
phi_avg 0.0011105990884470313
evaluating step 12
phi_avg 0.001358852221790102
evaluating step 0
phi_avg 0.0
evaluating step 1


c:\users\fahad\pycharmprojects\bmcs_shear_zone\bmcs_shear\dic_crack\dic_cor.py:142: RuntimeWarning: invalid value encountered in true_divide
  phi = 2 * np.arctan(len_d_0t / len_d_0c)


phi_avg 0.00021086525879563392
evaluating step 2
phi_avg 0.00041100276732816124
evaluating step 3
phi_avg 0.0002919294291647994
evaluating step 4
phi_avg 0.000353311447366805
evaluating step 5
phi_avg 0.0004383759120729785
evaluating step 6
phi_avg 0.0005660499323211656
evaluating step 7
phi_avg 0.0005550418412731924
evaluating step 8
phi_avg 0.0006307829223315434
evaluating step 9
phi_avg 0.0009064945062502224
evaluating step 10
phi_avg 0.0009347694239118565
evaluating step 11
phi_avg 0.0009915120798898744
evaluating step 0
phi_avg 0.0
evaluating step 1


c:\users\fahad\pycharmprojects\bmcs_shear_zone\bmcs_shear\dic_crack\dic_cor.py:142: RuntimeWarning: invalid value encountered in true_divide
  phi = 2 * np.arctan(len_d_0t / len_d_0c)


phi_avg 0.00015780810383966905
evaluating step 2
phi_avg 0.00021112823534240422
evaluating step 3
phi_avg 0.0001477434065904804
evaluating step 4
phi_avg 0.0002395654379415439
evaluating step 5
phi_avg 0.0004222986787743298
evaluating step 6
phi_avg 0.0005702746297894646
evaluating step 7
phi_avg 0.0006894711264261504
evaluating step 8
phi_avg 0.0008503166065295955
evaluating step 9
phi_avg 0.0009811383002989425
evaluating step 10
phi_avg 0.0010515005426457598


# B10_TV2

In [37]:
B10_TV2_cracks = [
    # crack 1
    (dict(y_ref_i=-1 , y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min = -11, n_x_max = -8, n_x_step=1, n_y_min = 1, n_y_max=5, n_y_step = 1)),
    # crack 2
    (dict(y_ref_i=-11, y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min = -16, n_x_max = -13, n_x_step=1, n_y_min = 1, n_y_max=4, n_y_step = 1)),
    # crack 3
    (dict(y_ref_i=-16, y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min = -21, n_x_max = -18, n_x_step=1, n_y_min = 1, n_y_max=5, n_y_step = 1)),
    # crack 4
    (dict(y_ref_i=-21, y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min = -29, n_x_max = -24, n_x_step=1, n_y_min = 1, n_y_max = 7, n_y_step = 1)),
    # crack 5
    (dict(y_ref_i=-29, y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min=-37, n_x_max=-33, n_x_step=1, n_y_min = 1, n_y_max=5, n_y_step = 1)),
     # crack 6
    (dict(y_ref_i=-37, y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min=0, n_x_max=14, n_x_step=1, n_y_min = 1, n_y_max=10, n_y_step = 1)),
]

In [38]:
dic_grid = DICGrid(n_x=54, n_y=13, d_x = 23, d_y = 23, start_t = 0, end_t =10, U_factor = 1, 
                   dir_name='B10_TV2', grid_column_first = False, grid_number_vertical = False)
dic_insp = DICInspector(dic_grid=dic_grid)
dic_insp.dic_cracks.items = construct_dic_cracks(B10_TV2_cracks)
dic_insp.interact()

evaluating step 0
phi_avg 0.0
evaluating step 1
phi_avg 0.00017545149186479097
evaluating step 2
phi_avg 0.0003913160348418573
evaluating step 3
phi_avg 0.0006368234345455437
evaluating step 4
phi_avg 0.0008729009125297093
evaluating step 5
phi_avg 0.001059374145230022
evaluating step 6
phi_avg 0.0013111483712865428
evaluating step 7
phi_avg 0.001638695402075936
evaluating step 8
phi_avg 0.0019059265938414533
evaluating step 9
phi_avg 0.001962270167191908
evaluating step 0


c:\users\fahad\pycharmprojects\bmcs_shear_zone\bmcs_shear\dic_crack\dic_cor.py:142: RuntimeWarning: invalid value encountered in true_divide
  phi = 2 * np.arctan(len_d_0t / len_d_0c)


phi_avg 0.0
evaluating step 1
phi_avg 0.0002191437146863288
evaluating step 2
phi_avg 0.000368141225260316
evaluating step 3
phi_avg 0.0004241326288931568
evaluating step 4
phi_avg 0.0004258725191167338
evaluating step 5
phi_avg 0.0007002040148946612
evaluating step 6
phi_avg 0.0006378608245061055
evaluating step 7
phi_avg 0.0007753762067577375
evaluating step 8
phi_avg 0.0008674212637481332
evaluating step 0
phi_avg 0.0
evaluating step 1
phi_avg 0.0003168467164471753
evaluating step 2
phi_avg 0.00039108681674453644
evaluating step 3
phi_avg 0.0002880506036932947
evaluating step 4
phi_avg 0.0004363420715320664
evaluating step 5
phi_avg 0.0006152172306577741
evaluating step 6
phi_avg 0.0007318803875277346
evaluating step 7
phi_avg 0.0008390322775951934
evaluating step 0
phi_avg 0.0
evaluating step 1
phi_avg 0.000125239960264578
evaluating step 2
phi_avg 0.0003331404076326799
evaluating step 3
phi_avg 0.0004191897694039965
evaluating step 4
phi_avg 0.0004248602542214293
evaluating step 5